4. Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости

In [1]:
life = [49,58,75,110,112,132,151,276,281,362]
import pandas as pd
life = pd.DataFrame(life)
medium = 200

In [2]:
from scipy.stats import wilcoxon

In [3]:
wilcoxon(life[0])

WilcoxonResult(statistic=0.0, pvalue=0.005062032126267864)

In [4]:
wilcoxon(life[0] - 200)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

5. Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

In [5]:
from scipy.stats import mannwhitneyu
nocut = [22,22,15,13,19,19,18,20,21,13,13,15]
cut = [17,18,18,15,12,4,14,15,10]
mannwhitneyu(nocut, cut, alternative = 'greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.02900499272087373)

6. С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? 

In [6]:
chal = pd.read_table('challenger.txt')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [7]:
chal.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [8]:
import numpy as np
np.random.seed(0)
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [9]:
bad = chal[chal.Incident == 1]
good = chal[chal.Incident == 0]

In [10]:
bad_sample = get_bootstrap_samples(bad.Temperature.values , 1000)
good_sample = get_bootstrap_samples(good.Temperature.values , 1000)

In [11]:
deltas = []
i=0
while i != 1000:
    deltas.append(bad_sample[i].mean() - good_sample[i].mean())
    i+=1
    
deltas[:3]

[-2.6339285714285694, -3.7633928571428577, -6.552678571428572]

In [12]:
stat_intervals(deltas, 0.05)

array([-8.06457589, -1.45040179])

7. На данных предыдущей задачи проверьте гипотезу об одинаковой средней температуре воздуха в дни, когда уплотнительный кольца повреждались, и дни, когда повреждений не было. Используйте перестановочный критерий и двустороннюю альтернативу. Чему равен достигаемый уровень значимости?

In [13]:
import itertools
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), list(filter(lambda i: i not in index, range(n)))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [14]:
bad_sample_means,  good_sample_means = [], []
i=0
while i != 1000:
    bad_sample_means.append(bad_sample[i].mean())
    good_sample_means.append(good_sample[i].mean())
    i+=1

In [15]:
permutation_t_stat_ind(bad_sample_means, good_sample_means)

-4.607176785714284

In [16]:
permutation_zero_dist_ind(bad_sample_means, good_sample_means, max_combinations = 10000)

[-0.09503214285714279,
 -0.05256428571427918,
 0.10016785714286414,
 0.09050892857143111,
 -0.013098214285708565,
 -0.016633928571422985,
 0.010664285714288013,
 -0.09293928571428367,
 0.04078035714286088,
 -0.2022624999999998,
 -0.1304857142857152,
 -0.10777142857143218,
 0.120478571428567,
 -0.1233678571428598,
 -0.07825357142857214,
 0.04287857142857732,
 -0.08806607142857459,
 0.18381607142857348,
 0.10233571428571153,
 0.032008928571428896,
 -0.15747321428571226,
 0.15238749999999968,
 -0.04387857142856788,
 0.2865660714285774,
 0.2160517857142814,
 0.0372249999999994,
 0.005789285714289605,
 -0.06749642857143101,
 0.12664821428571216,
 -0.01323571428571313,
 0.12114107142856767,
 0.06652857142857371,
 0.0826232142857144,
 -0.08311785714285946,
 0.21719464285714452,
 0.09128571428571419,
 -0.036910714285710355,
 -0.13803750000000292,
 -0.1824500000000029,
 -0.16240892857142697,
 0.011074999999994617,
 -0.10243035714286108,
 0.08394642857142998,
 0.363958928571428,
 0.1119107142857

In [27]:
np.random.seed(0)
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [30]:
np.random.seed(0)
permutation_test(good_sample_means, bad_sample_means, max_permutations = 10000, alternative = 'two-sided')

0.0001

Ответ 0.0057